In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
tf.set_random_seed(777)  # for reproducibility

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
xy = pd.read_csv('animal_for_tf.csv')
xy = xy.drop(['Unnamed: 0'], axis =1)

In [3]:
x_data = xy.drop(['newState'], axis = 1)
x_data = x_data.values
x_data

array([[0.9, 7. , 1. , ..., 0. , 0. , 0. ],
       [0.3, 6. , 1. , ..., 0. , 0. , 1. ],
       [3. , 6. , 6. , ..., 0. , 0. , 0. ],
       ...,
       [1.8, 4. , 0. , ..., 0. , 0. , 0. ],
       [5.3, 4. , 0. , ..., 0. , 0. , 0. ],
       [5.6, 4. , 2. , ..., 0. , 0. , 1. ]])

In [4]:
y_data = xy['newState'].values
y_data

array([0, 1, 3, ..., 1, 0, 1])

In [5]:
y_data = y_data.reshape(120481,1)

In [6]:
print(x_data.shape, y_data.shape)

print('Class label', np.unique(y_data))

nb_classes = 4

(120481, 11) (120481, 1)
Class label [0 1 2 3]


In [7]:
X = tf.placeholder(tf.float32, [None, 11])
Y = tf.placeholder(tf.int32, [None, 1])
print("reshape", Y.shape)  # 0 ~ 6

reshape (?, 1)


In [8]:
Y_one_hot = tf.one_hot(Y, nb_classes)  # one hot
print("reshape", Y_one_hot.shape)

reshape (?, 1, 4)


In [9]:
print("one_hot", Y_one_hot)
print("y_data.shape", Y_one_hot)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])
print("reshape", Y_one_hot.shape)

one_hot Tensor("one_hot:0", shape=(?, 1, 4), dtype=float32)
y_data.shape Tensor("one_hot:0", shape=(?, 1, 4), dtype=float32)
reshape (?, 4)


In [10]:
W1 = tf.get_variable("W1", shape=[11, 100],
                     initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([100]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

In [11]:
W2 = tf.get_variable("W2", shape=[100, 100],
                     initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([100]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

In [12]:
W3 = tf.get_variable("W3", shape=[100, 4],
                     initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([4]))
hypothesis = tf.matmul(L2, W3) + b3

In [13]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.0003).minimize(cost)

prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [15]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(501):
        sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
        if step % 10 == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={
                                 X: x_data, Y: y_data})
            print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(
                step, loss, acc))

    # Let's see if we can predict
    pred = sess.run(prediction, feed_dict={X: x_data})
    # y_data: (N,1) = flatten => (N, ) matches pred.shape

Step:     0	Loss: 16.008	Acc: 15.98%
Step:    10	Loss: 14.711	Acc: 16.02%
Step:    20	Loss: 13.819	Acc: 16.92%
Step:    30	Loss: 13.212	Acc: 18.27%
Step:    40	Loss: 12.730	Acc: 20.07%
Step:    50	Loss: 12.340	Acc: 21.23%
Step:    60	Loss: 12.067	Acc: 22.73%
Step:    70	Loss: 11.967	Acc: 26.71%
Step:    80	Loss: 12.030	Acc: 30.33%
Step:    90	Loss: 12.214	Acc: 33.20%
Step:   100	Loss: 12.473	Acc: 34.28%
Step:   110	Loss: 12.846	Acc: 34.31%
Step:   120	Loss: 13.435	Acc: 34.34%
Step:   130	Loss: 14.188	Acc: 34.47%
Step:   140	Loss: 15.125	Acc: 34.86%
Step:   150	Loss: 16.353	Acc: 35.00%
Step:   160	Loss: 17.931	Acc: 35.09%
Step:   170	Loss: 19.863	Acc: 35.13%
Step:   180	Loss: 22.159	Acc: 35.13%
Step:   190	Loss: 24.816	Acc: 35.11%
Step:   200	Loss: 27.807	Acc: 35.03%
Step:   210	Loss: 31.105	Acc: 34.98%
Step:   220	Loss: 34.695	Acc: 34.95%
Step:   230	Loss: 38.565	Acc: 34.90%
Step:   240	Loss: 42.703	Acc: 34.86%
Step:   250	Loss: 47.100	Acc: 34.82%
Step:   260	Loss: 51.753	Acc: 34.82%
S